In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, Random, Distributions, LinearAlgebra
import Plots: Plot
import JuliaProbo: AbstractObject, Landmark, Map, KalmanFilter, EstimatorAgent, RealRobot, RealCamera, World, Goal, observations, push!, draw,decision, state_transition
gr();

In [ ]:
struct Puddle <: AbstractObject
    lowerleft::Vector{Float64}
    upperright::Vector{Float64}
    depth::Float64
    function Puddle(lowerleft::Vector{Float64}, upperright::Vector{Float64}, depth::Float64)
        new(lowerleft, upperright, depth)
    end
end

function draw(puddle::Puddle, p::Plot{T}) where {T}
    w = puddle.upperright[1] - puddle.lowerleft[1]
    h = puddle.upperright[2] - puddle.lowerleft[2]
    x = puddle.lowerleft[1]
    y = puddle.lowerleft[2]
    shape_x, shape_y = [x, x+w, x+w, x], [y, y, y+h, y+h]
    shapes = [(shape_x[i], shape_y[i]) for i in 1:4]
    push!(shapes, (shape_x[1], shape_y[1]))
    p = plot!(shapes, seriestype=:shape, fillcolor=:blue, opacity=0.15)
end

function ch10_puddle_world2()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks =
        [Landmark([-4.0, 2.0], 0), Landmark([2.0, -3.0], 1), Landmark([3.0, 3.0], 2)]
    envmap = Map()
    push!(envmap, landmarks)
    world = World(xlim, ylim)
    push!(world, envmap)
    # robot side
    initial_pose = [0.0, 0.0, 0.0]
    estimator = KalmanFilter(envmap, initial_pose)
    agent = EstimatorAgent(0.2, 10.0 * pi / 180, dt, estimator)
    robot = RealRobot(initial_pose, agent, RealCamera(landmarks, distance_bias_rate_stddev = 0.0, direction_bias_stddev = 0.0); color = "red")
    push!(world, robot)
    # goal
    goal = Goal(-3.0, -3.0)
    push!(world, goal)
    # puddles
    push!(world, Puddle([-2.0, 0.0], [0.0, 2.0], 0.1))
    push!(world, Puddle([-0.5, -2.0], [2.5, 1.0], 0.1))
    
    anim = @animate for i = 1:300
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        p = draw(world, annota)
        z = observations(robot.sensor_, robot.pose_; noise = false, bias = false)
        v, ω = decision(agent, z, envmap)
        state_transition(robot, v, ω, dt; move_noise = true, vel_bias_noise = true)
    end
    gif(anim, "images/ch10_puddle_world2.gif", fps = 20)
end
ch10_puddle_world2()

<img src="images/ch10_puddle_world2.gif">